<a href="https://colab.research.google.com/github/gokulanv/LeNet_Pytorch/blob/master/PytorchBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms

In [2]:
from torch import FloatTensor
from torch.autograd import Variable


# Define the leaf nodes
a = Variable(FloatTensor([4]))

weights = [Variable(FloatTensor([i]), requires_grad=True) for i in (2, 5, 9, 7)]

# unpack the weights for nicer assignment
w1, w2, w3, w4 = weights

b = w1 * a #8
print(w1)
c = w2 * a #20
d = w3 * b + w4 * c # 72 + 140
L = (10 - d) # 202

L.backward()

for index, weight in enumerate(weights, start=1):
    gradient, *_ = weight.grad.data
    print(f"Gradient of w{index} w.r.t to L: {gradient}")

print(weights[0].grad)

tensor([2.], requires_grad=True)
Gradient of w1 w.r.t to L: -36.0
Gradient of w2 w.r.t to L: -28.0
Gradient of w3 w.r.t to L: -8.0
Gradient of w4 w.r.t to L: -20.0
tensor([-36.])


In [3]:

# ================================================================== #
#                         Table of Contents                          #
# ================================================================== #

# 1. Basic autograd example 1               (Line 25 to 39)
# 2. Basic autograd example 2               (Line 46 to 83)
# 3. Loading data from numpy                (Line 90 to 97)
# 4. Input pipline                          (Line 104 to 129)
# 5. Input pipline for custom dataset       (Line 136 to 156)
# 6. Pretrained model                       (Line 163 to 176)
# 7. Save and load model                    (Line 183 to 189) 


# ================================================================== #
#                     1. Basic autograd example 1                    #
# ================================================================== #

# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 


# ================================================================== #
#                    2. Basic autograd example 2                     #
# ================================================================== #

# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)

# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

# Backward pass.
loss.backward()

# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())


tensor(2.)
tensor(1.)
tensor(1.)
w:  Parameter containing:
tensor([[ 0.4113, -0.4054, -0.3912],
        [ 0.1290, -0.4298, -0.5543]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.3056, -0.3107], requires_grad=True)
loss:  0.9141187071800232
dL/dw:  tensor([[ 0.0096, -0.0418, -0.2185],
        [ 0.3297, -0.0041, -0.4163]])
dL/db:  tensor([ 0.4149, -0.6936])
loss after 1 step optimization:  0.9043300747871399


In [4]:
# ================================================================== #
#                         4. Input pipline                           #
# ================================================================== #

# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of the data loader is as below.
c = 0
for images, labels in train_loader:
    # Training code should be written here.
    if c < 10:
      print(labels)
      c = c + 1
    pass


0it [00:00, ?it/s]

170500096it [00:06, 26502993.30it/s]                               


Extracting ../../data/cifar-10-python.tar.gz to ../../data/
torch.Size([3, 32, 32])
6
tensor([3, 6, 2, 3, 4, 1, 1, 0, 3, 6, 2, 3, 4, 4, 0, 5, 3, 8, 2, 6, 6, 7, 4, 0,
        9, 8, 7, 9, 0, 4, 7, 9, 3, 1, 9, 6, 6, 6, 9, 7, 6, 7, 0, 5, 3, 2, 9, 6,
        6, 5, 6, 7, 1, 1, 8, 8, 8, 7, 7, 2, 5, 4, 5, 6])
tensor([4, 8, 5, 7, 7, 9, 6, 3, 6, 8, 2, 5, 4, 5, 0, 4, 4, 1, 8, 0, 7, 0, 7, 1,
        1, 1, 1, 3, 8, 2, 3, 2, 3, 8, 9, 6, 6, 3, 2, 2, 2, 0, 7, 5, 7, 6, 4, 2,
        0, 8, 2, 9, 6, 5, 9, 3, 5, 6, 6, 5, 2, 2, 5, 1])
tensor([1, 3, 6, 7, 8, 6, 1, 4, 7, 4, 6, 0, 7, 4, 3, 7, 2, 0, 8, 3, 5, 3, 2, 3,
        6, 6, 7, 9, 0, 3, 2, 5, 8, 0, 3, 2, 4, 6, 8, 7, 2, 6, 0, 6, 6, 6, 1, 4,
        7, 1, 1, 1, 7, 6, 5, 6, 1, 1, 9, 0, 8, 0, 9, 1])
tensor([9, 8, 4, 0, 6, 3, 2, 7, 1, 8, 7, 2, 4, 9, 1, 6, 3, 8, 3, 6, 5, 3, 7, 6,
        1, 1, 2, 2, 6, 6, 2, 6, 1, 0, 2, 4, 6, 9, 4, 0, 2, 8, 7, 6, 1, 0, 9, 1,
        5, 2, 4, 7, 1, 1, 9, 0, 5, 8, 2, 0, 5, 1, 5, 2])
tensor([7, 0, 2, 4, 2, 8, 2, 6, 1, 4, 2, 8, 0,

In [5]:
c = 0
for images, labels in train_loader:
    # Training code should be written here.
    if c < 10:
      print(images[0].size())
      c = c + 1
    pass

torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])


In [0]:

# ================================================================== #
#                5. Input pipline for custom dataset                 #
# ================================================================== #

# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)


# ================================================================== #
#                        6. Pretrained model                         #
# ================================================================== #

# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)


# ================================================================== #
#                      7. Save and load the model                    #
# ================================================================== #

# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))